<a href="https://colab.research.google.com/github/squeeko/TensorflowProjects/blob/in_progress/TF_LoadText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load text

This tutorial provides an example of how to use [tf.data.TextLineDataset](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) to load examples from text files. TextLineDataset is designed to create a dataset from a text file, in which each example is a line of text from the original file. This is potentially useful for any text data that is primarily line-based (for example, poetry or error logs).

In this tutorial, we'll use three different English translations of the same work, Homer's Illiad, and train a model to identify the translator given a single line of text.

## Setup

In [23]:
pip install -q tfds-nightly

In [24]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

The texts of the three translations are by:



*   William Cowper — text

*   Edward, Earl of Derby — text

*   Samuel Butler — text

The text files used in this tutorial have undergone some typical preprocessing tasks, mostly removing stuff — document header and footer, line numbers, chapter titles. Download these lightly munged files locally.

In [25]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

'/root/.keras/datasets'

## Load text into datasets

Iterate through the files, loading each one into its own dataset.

Each example needs to be individually labeled, so use [tf.data.Dataset.map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) to apply a labeler function to each one. This will iterate over every example in the dataset, returning (example, label) pairs.

In [26]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [27]:
labeled_dataset

<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>

Combine these labeled datasets into a single dataset, and shuffle it.

In [28]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [29]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False
)


You can use [tf.data.Dataset.take](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) and print to see what the (example, label) pairs look like. The numpy property shows each Tensor's value.

In [30]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b"As morn, in saffron robe, o'er all the earth">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'broad river Axius and Periboea eldest daughter of Acessamenus; for the'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"By closest scrutiny, which way are driv'n">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"With fiery speed; for on that race was stak'd">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Him rescued even on the verge of fate.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


## Encode text lines as numbers

Machine learning models work on numbers, not words, so the string values need to be converted into lists of numbers. To do that, map each unique word to a unique integer.

### Build vocabulary

First, build a vocabulary by tokenizing the text into a collection of individual unique words. There are a few ways to do this in both TensorFlow and Python. For this tutorial:



1.   Iterate over each example's numpy value.
2.   Use [tfds.deprecated.text.Tokenizer](https://www.tensorflow.org/datasets/api_docs/python/tfds/deprecated/text/Tokenizer) to split it into tokens.
3.   Collect these tokens into a Python set, to remove duplicates.
4.   Get the size of the vocabulary for later use.

In [32]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

  vocab_size = len(vocabulary_set)


In [33]:
vocab_size

17178

## Encode examples

Create an encoder by passing the vocabulary_set to [tfds.deprecated.text.TokenTextEncoder](https://www.tensorflow.org/datasets/api_docs/python/tfds/deprecated/text/TokenTextEncoder). The encoder's encode method takes in a string of text and returns a list of integers.

In [35]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

You can try this on a single line to see what the output looks like.



In [36]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b"As morn, in saffron robe, o'er all the earth"


In [37]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[10611, 16808, 2604, 11958, 11850, 5604, 13847, 12346, 865, 11145]


Now run the encoder on the dataset by wrapping it in [tf.py_function](https://www.tensorflow.org/api_docs/python/tf/py_function) and passing that to the dataset's map method.

In [38]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

You want to use [Dataset.map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) to apply this function to each element of the dataset. Dataset.map runs in graph mode.

*   Graph tensors do not have a value.
*   In graph mode you can only use TensorFlow Ops and functions.

So you can't .map this function directly: You need to wrap it in a tf.py_function. The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function.

In [39]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode,
                                       inp = [text, label],
                                       Tout = (tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually:  
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label 

all_encoded_data = all_labeled_data.map(encode_map_fn)



## Split the dataset into test and train batches

Use [tf.data.Dataset.take](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) and  [tf.data.Dataset.skip](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) to create a small test dataset and a larger training set.

Before being passed into the model, the datasets need to be batched. Typically, the examples inside of a batch need to be the same size and shape. But, the examples in these datasets are not all the same size — each line of text had a different number of words. So use [tf.data.Dataset.padded_batch](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#padded_batch) (instead of batch) to pad the examples to the same size.

In [40]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

Now, test_data and train_data are not collections of (example, label) pairs, but collections of batches. Each batch is a pair of (many examples, many labels) represented as arrays.

To illustrate:

In [41]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(17,), dtype=int64, numpy=
 array([10611, 16808,  2604, 11958, 11850,  5604, 13847, 12346,   865,
        11145,     0,     0,     0,     0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

Since we have introduced a new token encoding (the zero used for padding), the vocabulary size has increased by one.

In [42]:
vocab_size += 1

## Build the model

In [43]:
model = tf.keras.Sequential()

The first layer converts integer representations to dense vector embeddings. See the [word embeddings tutorial](https://www.tensorflow.org/tutorials/text/word_embeddings) for more details.

In [44]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

The next layer is a [Long Short-Term Memory](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) layer, which lets the model understand words in their context with other words. A bidirectional wrapper on the LSTM helps it to learn about the datapoints in relationship to the datapoints that came before it and after it.

In [46]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

Finally we'll have a series of one or more densely connected layers, with the last one being the output layer. The output layer produces a probability for all the labels. The one with the highest probability is the models prediction of an example's label.

In [47]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.

for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

Finally, compile the model. For a softmax categorization model, use sparse_categorical_crossentropy as the loss function. You can try other optimizers, but adam is very common.

In [48]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

## Train the model

This model running on this data produces decent results (about 83%).

In [49]:
model.fit(
    train_data,
    epochs = 3,
    validation_data = test_data
)

Epoch 1/3
697/697 [==============================] - 27s 39ms/step - loss: 0.5122 - accuracy: 0.7467 - val_loss: 0.3790 - val_accuracy: 0.8238
Epoch 2/3
697/697 [==============================] - 27s 39ms/step - loss: 0.2951 - accuracy: 0.8717 - val_loss: 0.3554 - val_accuracy: 0.8380
Epoch 3/3
697/697 [==============================] - 27s 38ms/step - loss: 0.2240 - accuracy: 0.9039 - val_loss: 0.3653 - val_accuracy: 0.8442


In [50]:
eval_loss, eval_acc = model.evaluate(test_data)
print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))


79/79 [==============================] - 2s 25ms/step - loss: 0.3653 - accuracy: 0.8442

Eval loss: 0.365, Eval accuracy: 0.844
